##### IsaacSim API Tutorial (GUI)

- Visualize Script Editor window `(Window -> Script Editor)`
  
- The following script is intended to be run inside the Script

  - (주의!) 스크립트를 복붙하기 전에 콘솔창을 깨끗이하고 붙여넣기


##### 1. Initialize World

- `World` is the core class that enables you to interact with the simulator in an easy and modular way
- callbacks, stepping physics, resetting the scene, adding tasks, etc.
- Singleton Class; One `World` per One Isaac Sim


In [ ]:
# Inside docker container set nucleus default path to reduce checking server time
import carb
import omni.isaac.core.utils.carb as carb_utils
settings = carb.settings.get_settings()
carb_utils.set_carb_setting(settings, "/persistent/isaac/asset_root/default", "http://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/2023.1.1")

In [ ]:
# Initialize World
from omni.isaac.core import World
world = World(stage_units_in_meters=1.0)
world.scene.add_default_ground_plane()

##### 2. Add and Remove Objects

- Isaac Sim에서는 기본적으로 몇가지 Primitive Shape 물체를 제공하고 있음
- `omni.isaac.core.objects` 에서 확인 가능하며
    - Cuboid
    - Cone
    - Cylinder
    - Sphere
    - Capsule
- 물체 타입은 아래와 같은데, 직접 실습으로 익히는게 빠릅니다.
  
    - `Visual`: 랜더링만 가능(물리적인 특성이 없음) (다른 물체를 통과 시킴)
    - `Fixed`: `Visual` 에서 다른 물체와 상호작용이 가능(물체가 통과하지 않음, 예시: 바닥)
    - `Dynamic`: 질량, 마찰 등의 물리적인 특징을 보유
- 위 3가지로 `Visual`->`Fixed`->`Dynamic` 순으로 상속 관계를 가짐
    ```python
    class VisualCuboid(GeometryPrim):
        ...
    class FixedCuboid(VisualCuboid):
        ...
    class DynamicCuboid(RigidPrim, FixedCuboid):
        ...
    ```

- (주의!) Isaac Sim 내에 물체를 선언할 때에는 `prim_path` 및 `name` 값은 물체 간에 겹치지 않도록 선언해주어야 함



In [ ]:
# Add Object

from omni.isaac.core.objects.cuboid import DynamicCuboid, FixedCuboid, VisualCuboid
import numpy as np

# Red Cube:: Visual Cuboid -> 시각적으로만 표현
cube1 = world.scene.add(VisualCuboid(prim_path="/World/visual",
                    name="visual_cube",
                    position=[0.0, 0.0, 0.5], 
                    scale=[0.1, 0.1, 0.1],
                    color=np.array([1.0, 0.0, 0.0])))

# Green Cube:: Fixed Cuboid -> 중력에 영향을 받지 않음
cube2 = world.scene.add(FixedCuboid(prim_path="/World/fixed",
                   name="fixed_cube",
                   position=[0.3, 0.0, 0.5], 
                   scale=[0.1, 0.1, 0.1], 
                   color=np.array([0.0, 1.0, 0.0])))

# Blue Cube:: Dynamic Cuboid -> 중력에 영향을 받음
cube3 = world.scene.add(DynamicCuboid(prim_path="/World/dynamic1", 
                     name='dynamic_cube',
                     position=[0.5, 0.0, 0.5], 
                     scale=[0.1, 0.1, 0.1], 
                     color=np.array([0.0, 0.0, 1.0])))

In [ ]:
# Clear All contents
world.clear()

In [ ]:

# Add Object
world.scene.add_default_ground_plane()
cube = world.scene.add(VisualCuboid(prim_path="/World/visual",
                    name="visual_cube",
                    position=[0.0, 0.0, 0.5], 
                    scale=[0.1, 0.1, 0.1],
                    color=np.array([1.0, 0.0, 0.0])))



In [ ]:

# Control Object
cube.set_visibility(False) # True로 변경하면 다시 보임

cube.set_world_pose([1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0])

cube.set_world_pose([1.0, 0.0, 0.05], [0.0, 0.0, 0.0, 1.0])

# Remove Object
world.scene.remove_object(cube.name)

##### 3. Set hierarchy with `prim_path`

In [ ]:
# Clear All contents
world.clear()
world.scene.add_default_ground_plane()

In [ ]:
from omni.isaac.core.prims import XFormPrim

# Add x prim
x = world.scene.add(XFormPrim("/World/X", name='x_xform'))

# Add Object under x prim
cube = world.scene.add(VisualCuboid(prim_path="/World/X/visual",
                        name="visual_cube_x",
                        position=[0.0, 0.0, 0.5], 
                        scale=[0.1, 0.1, 0.1],
                        color=np.array([1.0, 0.0, 0.0])))


# x prim의 특성을 변경하면 하위 오브젝트도 같이 변경
x.set_visibility(False) # True로 변경하면 다시 보임

x.set_world_pose([1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]) # x prim의 위치 변경하면 하위 오브젝트도 같이 변경

world.scene.remove_object(x.name) # x prim을 삭제하면 하위 오브젝트도 같이 삭제

##### 4. Play with Camera Sensor

In [ ]:
# Clear All contents
world = World(stage_units_in_meters=1.0)
world.scene.add_default_ground_plane()

In [ ]:
from omni.isaac.sensor import Camera

my_camera = Camera(                                             
    prim_path="/World/RGB",                                     
    frequency=20,                                               
    resolution=(640, 480),                                    
    position=[0.0, 0.0, 1.0],  
    orientation=[0.5,-0.5,0.5,0.5]                
)
my_camera.initialize()                                          
my_camera.set_focal_length(1.93)                                
my_camera.set_focus_distance(4)                                 
my_camera.set_horizontal_aperture(2.65)                         
my_camera.set_vertical_aperture(1.48)                           
my_camera.set_clipping_range(0.01, 10000)

- `Window` -> `Viewport` 로 가서 하나는 default 카메라 뷰, 다른 하나는 생성된 카메라 뷰
- 화면의 좌측 상단에 카메라 모양을 눌러서 `Cameras` -> `RGB`

In [ ]:
from omni.isaac.core.objects.cuboid import DynamicCuboid, FixedCuboid, VisualCuboid
import numpy as np

# Red Cube:: Visual Cuboid -> 시각적으로만 표현
cube1 = world.scene.add(VisualCuboid(prim_path="/World/visual",
                    name="visual_cube",
                    position=[0.0, 0.0, 0.5], 
                    scale=[0.1, 0.1, 0.1],
                    color=np.array([1.0, 0.0, 0.0])))
# Green Cube:: Fixed Cuboid -> 중력에 영향을 받지 않음
cube2 = world.scene.add(FixedCuboid(prim_path="/World/fixed",
                   name="fixed_cube",
                   position=[0.3, 0.0, 0.5], 
                   scale=[0.1, 0.1, 0.1], 
                   color=np.array([0.0, 1.0, 0.0])))


# Blue Cube:: Dynamic Cuboid -> 중력에 영향을 받음
cube3 = world.scene.add(DynamicCuboid(prim_path="/World/dynamic1", 
                     name='dynamic_cube',
                     position=[0.5, 0.0, 0.5], 
                     scale=[0.1, 0.1, 0.1], 
                     color=np.array([0.0, 0.0, 1.0])))

- 카메라 모양 우측에 센서 출력 메뉴를 눌러서 윈도우를 시각화(메뉴 클릭 후 -> `Show Window`)
- RGB 이미지를 포함한 다양한 데이터를 관측할 수 있음
- 아래 코드는 Semantic Segmenation 라벨을 정하는 코드

In [ ]:
# Add Distance and Instance Segmentation

my_camera.add_distance_to_camera_to_frame()            
my_camera.add_instance_segmentation_to_frame()

In [ ]:
# Set each object's semantics
from omni.isaac.core.utils.semantics import add_update_semantics

visual_cube = world.scene.get_object('visual_cube')
fixed_cube = world.scene.get_object('fixed_cube')
dynamic_cube = world.scene.get_object('dynamic_cube')

add_update_semantics(visual_cube.prim, "0")
add_update_semantics(fixed_cube.prim, "1")
add_update_semantics(dynamic_cube.prim, "2")